This notebook contains the code and all my understandings and everything about magic. This is somehow my first serious work in machine learning.

In [1]:
import torch
# TODO: make these constants arguments
n_route = 228
n_hist = 12
n_pred = 9
batch_size = 50
epoch = 50
# parser.add_argument('--save', type=int, default=10)
ks = 3  # spatial convolution kernel size
kt = 3  # temporal convolution kernel size
lr = 1e-3
opt = torch.optim.RMSprop
# parser.add_argument('--graph', type=str, default='default')
# parser.add_argument('--inf_mode', type=str, default='merge')

The model has two spatio temporal blocks and each block has two gated temporal convolutional layers and one spatial convolutional layer in between so we have totaly six layers and we should determine the size of channel for these layers

In [5]:
blocks = [[1, 32, 64], [64, 32, 128]]

We should have a file which defines the graph we want to work on.

In [46]:
import pandas as pd

W = pd.read_csv("PeMSD7_W_228.csv", header=None)
W_tensor = torch.tensor(W.values)
W_tensor.shape

torch.Size([228, 228])

Equation 10 on original papaer

In [59]:
def weight_matrix(W, sigma2=0.1, epsilon=0.5, scaling=True):
    '''
    Weight matrix function.
    :param W: tensor, the adjacency matrix of the graph.
    :param sigma2: float, scalar of matrix W.
    :param epsilon: float, thresholds to control the sparsity of matrix W.
    :param scaling: bool, whether applies numerical scaling on W.
    :return: np.ndarray, [n_route, n_route].
    '''
    # check whether W is a 0/1 matrix.
    if set(torch.unique(W)) == {0, 1}:
        print('The input graph is a 0/1 matrix; set "scaling" to False.')
        scaling = False

    if scaling:
        n = W.shape[0]
        W = W / 10000. # WHY
        W2, W_mask = W * W, np.ones([n, n]) - np.identity(n)
        # refer to Eq.10
        a = torch.exp(-W2 / sigma2)
        a[a >= epsilon] = 0
        
        return a * W_mask
    
    return W

In [60]:
tmp = weight_matrix(W_tensor)
print(torch.unique(tmp))

tensor([0.0000e+00, 1.4279e-89, 1.0224e-88,  ..., 4.9829e-01, 4.9897e-01,
        4.9999e-01], dtype=torch.float64)


In the below cell I want to calculate normalized laplacian.<br/>
I calculate it using \(L=I - D^(-1/2) W D^(-1/2)\)

In [20]:
import numpy as np

def normalized_laplacian(W):
    '''
    Normalized graph Laplacian function.
    :param W: np.ndarray, [n_route, n_route], weighted adjacency matrix of G.
    :return: np.matrix, [n_route, n_route].
    '''
    # d ->  diagonal degree matrix
    n, d = np.shape(W)[0], torch.sum(W, axis=1)
    I = torch.eye(n)
    
    # d^(-1/2)
    d = torch.pow(d, -1/2)
    # make d diagonal
    d = torch.diag(d)

    dw = torch.matmul(d, W)
    dwd = torch.matmul(dw, d)
    
    return dwd

In [21]:
T = torch.tensor([[3,5,8], [5, 8, 3], [8, 3, 5]], dtype=float)
print(normalized_laplacian(T))

<class 'torch.Tensor'>
+++++
<class 'torch.Tensor'>
tensor([[0.1875, 0.3125, 0.5000],
        [0.3125, 0.5000, 0.1875],
        [0.5000, 0.1875, 0.3125]], dtype=torch.float64)


**Chebyshev polynomials approximation**

In [22]:
import pandas as pd

data_file = pd.read_csv("PeMSD7_V_228.csv", header=None)
data_file_tensor = torch.tensor(data_file.values)
data_file_tensor

tensor([[68.1000, 66.8000, 61.7000,  ..., 67.7000, 68.8000, 68.8000],
        [68.0000, 64.3000, 66.6000,  ..., 70.2000, 69.1000, 68.7000],
        [68.3000, 67.8000, 65.9000,  ..., 70.4000, 67.3000, 69.0000],
        ...,
        [68.9000, 37.9000, 68.8000,  ..., 68.7000, 67.5000, 19.7000],
        [69.2000, 37.8000, 68.7000,  ..., 67.7000, 67.4000, 19.5000],
        [68.6000, 52.9000, 68.6000,  ..., 68.1000, 68.5000, 19.1000]],
       dtype=torch.float64)

In [61]:
n_train, n_val, n_test = 34, 5, 5